<a href="https://colab.research.google.com/github/NodiraTillayeva/ERA5Analysis/blob/main/ParseNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parsing netcdf files with python


In [1]:
%pip install numpy pandas xarray netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.0 MB/s eta 0:00:00


In [ ]:
%pip install matplotlib geopandas

In [ ]:
import netCDF4 as nc
import pandas as pd
from datetime import datetime

In [ ]:
# Replace with your .nc file path
nc_file = '/content/travka.nc'

In [ ]:
# Open the NetCDF file
ds = nc.Dataset(nc_file)

In [ ]:
# List all variables
print("Variables in the file:")
for var in ds.variables:
    print(var)

Variables in the file:
longitude
latitude
expver
time
lai_hv
lai_lv


In [ ]:
# Extract data
# Assuming variable names are 'time', 'latitude', 'longitude', 'low_veg_index', 'high_veg_index'
# Modify as per your file's variable names
times = ds.variables['time'][:]
lats  = ds.variables['latitude'][:]
lons  = ds.variables['longitude'][:]
low_veg  = ds.variables['lai_lv'][:]
high_veg  = ds.variables['lai_hv'][:]

In [ ]:
# Prepare a list to hold the data
data = []

# Iterate over each time, latitude, and longitude
for time_index, time_value in enumerate(times):
    for lat_index, lat_value in enumerate(lats):
        for lon_index, lon_value in enumerate(lons):
            # Check if indices are within the bounds of the low_veg and high_veg arrays
            if lat_index < low_veg.shape[1] and lon_index < low_veg.shape[2]:
                # Extracting values for low and high vegetation at this point
                low_veg_value = low_veg[time_index, lat_index, lon_index]
                high_veg_value = high_veg[time_index, lat_index, lon_index]

                # Appending to the data list
                data.append([time_value, lat_value, lon_value, low_veg_value, high_veg_value])

# Convert to DataFrame
df = pd.DataFrame(data, columns=['Time', 'Latitude', 'Longitude', 'Low Vegetation', 'High Vegetation'])

# Save to CSV
csv_file_path = 'output_data.csv'
df.to_csv(csv_file_path, index=False)

# Close the NetCDF file
dataset.close()

In [ ]:
ds.close()

In [ ]:
df.tail()

,Time,Latitude,Longitude,Low Vegetation,High Vegetation
150615,1084752,45.48,63.99,1.318490,[2.098176339234622 2.097805767645213 1.8193582...
150616,1084752,45.48,64.09,1.219858,[1.7616973360511117 0.5423685782592356 2.04488...
150617,1084752,45.48,64.19,1.285331,[2.1676955694077793 1.2752850677926577 1.25008...
150618,1084752,45.48,64.29,1.129772,[1.9352730685303587 1.927268722199121 1.803942...
150619,1084752,45.48,64.39,0.854929,[-4.440892098500626e-16 -4.440892098500626e-16...
